# Pipeline

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from WorkforceSentimentMonitoring.data import get_prepaired_data
from WorkforceSentimentMonitoring.encoders import Preprocessor, CustomMinMaxScaler, FeatureEngineer
from WorkforceSentimentMonitoring.preprocessing import lemmatize
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import f1_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, LogisticRegression

from langdetect import detect

## Get data

In [32]:
X_train, X_test, y_train, y_test = get_prepaired_data()

Reading data...
Merging data into a single DataFrame...
Identifying entries in other languages...
Drop 436 entries? y / [n]
y
Dropping 436 entries...


NameError: name 'inplace' is not defined

In [ ]:
X_train['review'].apply(detect)

## Preprocess

In [29]:
preprocessor = Preprocessor()
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.fit_transform(X_test)

In [30]:
X_train.head(1)

,summary,positives,negatives,advice_to_mgmt,review
0,brand specialist,high learning curve high independent,people management work life balance,NaN,brand specialist high learning curve high inde...


## Feature Engineering

In [25]:
engineer = FeatureEngineer()
X_train = engineer.fit_transform(X_train)
X_test = engineer.fit_transform(X_test)

100%|██████████| 5/5 [00:00<00:00, 36.35it/s]


In [31]:
display(X_train.head(1))
display(X_test.head(1))

,summary,positives,negatives,advice_to_mgmt,review
0,brand specialist,high learning curve high independent,people management work life balance,NaN,brand specialist high learning curve high inde...


,summary,positives,negatives,advice_to_mgmt,review
0,great stepping stone,amazing benefit and employee right everything...,apple talk big about internal growth and devel...,promote internally employee to management you...,great stepping stone amazing benefit and emplo...


## Pipeline

In [ ]:
pipe = Pipeline([
    ('preprocessor', Preprocessor()),
    ('engineer', FeatureEngineer()),
    ('scaler', CustomMinMaxScaler()),
    ()
])

# Extra: Implementation wrong language function

In [ ]:
def drop_wrong_language(df, column, language = 'en', inplace=False):
    '''drops entries written in languages other thatn the specified'''
    print('Identifying entries in other languages...')
    is_wrong = df[column].apply(detect) != language
    n_rows_to_drop = is_wrong.sum()

    user_confirmation = None
    while not (user_confirmation is 'y' or user_confirmation is 'n'):
        user_confirmation = input(f'Drop {n_rows_to_drop} entries? y / [n]\n') or 'n'
    if user_confirmation is 'y':
        if inplace:
            print(f'Dropping {n_rows_to_drop} entries...')
            df = df[~is_wrong]
            df.reset_index(inplace=True, drop=True)
            print('Process completed.')
            return df
        else:
            print(f'Dropping {n_rows_to_drop} entries...')
            print('Process completed.')
            return df[~is_wrong].reset_index(inplace=True, drop=True)
    else:
        print('Process aborted')
        return None